In [1]:
from data.utils.lmdb_utils import LMDBDataset

dataset = LMDBDataset('data/graphs/QM9_train.lmdb')

In [2]:
graph, target = dataset[1]

print(graph)
print(target)

{'atom_atomic_nums': array([5, 5, 5, 7, 5, 7, 5], dtype=uint8), 'edge_index': array([[0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 1, 6, 2],
       [1, 0, 2, 1, 3, 2, 4, 3, 5, 4, 6, 5, 1, 6, 2, 6]], dtype=uint8), 'edge_attr': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)}
[ 4.57111000e+00  2.64766000e+00  2.51580000e+00  1.10280000e+00
  5.61400000e+01 -2.45900000e-01  9.22000000e-02  3.38100000e-01
  6.24804200e+02  1.25739000e-01 -3.45627428e+02 -3.45621103e+02
 -3.45620159e+02 -3.45657610e+02  2.32890000e+01 -1.41956718e+03
 -1.42892961e+03 -1.43722779e+03 -1.32062093e+03]
